In [4]:
pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


In [3]:
import keras

In [4]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SUHAS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SUHAS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file) #data_file will be in the form of text so load it as json

In [6]:
#data_file

In [7]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me',
    'you are great!'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'botstatus',
   'patterns': ['How are you doing?', 

In [8]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [9]:
print(classes)

['greeting', 'goodbye', 'thanks', 'botstatus', 'creator', 'sad', 'funny', 'positive', 'options', 'adverse_drug', 'blood_pressure', 'blood_pressure_search', 'pharmacy_search', 'hospital_search']


In [10]:
# lemmaztize and lower each word and remove duplicates
#lemmatizing is a method of identifying the base word for example caring stemming gives car but lemmatizing gives care 
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]


In [11]:
print(words)

['hi', 'there', 'how', 'are', 'you', 'is', 'anyone', 'there', 'hey', 'hola', 'hello', 'good', 'day', 'bye', 'see', 'you', 'later', 'goodbye', 'nice', 'chatting', 'to', 'you', ',', 'bye', 'till', 'next', 'time', 'thanks', 'thank', 'you', 'that', "'s", 'helpful', 'awesome', ',', 'thanks', 'thanks', 'for', 'helping', 'me', 'you', 'are', 'great', 'how', 'are', 'you', 'doing', 'whats', 'up', 'are', 'you', 'fine', 'who', 'are', 'you', 'who', 'is', 'this', 'who', 'am', 'i', 'talking', 'to', 'what', 'is', 'your', 'name', 'are', 'you', 'human', 'i', 'am', 'sad', 'i', 'am', 'upset', 'i', 'have', 'a', 'problem', 'i', 'am', 'not', 'okay', 'bad', 'day', 'feeling', 'low', 'you', 'have', 'good', 'sense', 'of', 'humour', 'you', 'are', 'funny', 'bot', 'ok', 'good', 'cool', 'yes', 'wow', 'you', 'are', 'so', 'intelligent', 'hurray', 'i', 'am', 'happy', 'i', 'am', 'doing', 'good', 'damn', 'how', 'you', 'could', 'help', 'me', 'what', 'you', 'can', 'do', 'what', 'help', 'you', 'provide', 'how', 'you', 'can'

In [12]:
words1 = sorted(list(set(words))) #duplicates are removed
print(words1)

["'s", ',', 'a', 'adverse', 'all', 'am', 'anyone', 'are', 'awesome', 'bad', 'be', 'behavior', 'blood', 'bot', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'cool', 'could', 'damn', 'data', 'day', 'detail', 'do', 'doing', 'dont', 'drug', 'entry', 'feeling', 'find', 'fine', 'for', 'funny', 'give', 'good', 'goodbye', 'great', 'happy', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'human', 'humour', 'hurray', 'i', 'id', 'intelligent', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'low', 'management', 'me', 'module', 'name', 'nearby', 'next', 'nice', 'not', 'of', 'offered', 'ok', 'okay', 'open', 'patient', 'pharmacy', 'pressure', 'problem', 'provide', 'reaction', 'related', 'result', 'sad', 'search', 'searching', 'see', 'sense', 'show', 'so', 'suitable', 'support', 'talking', 'task', 'thank', 'thanks', 'that', 'there', 'this', 'till', 'time', 'to', 'transfer', 'up', 'upset', 'want', 'what', 'whats', 'which', 'w

In [13]:
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

70 documents
14 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'botstatus', 'creator', 'funny', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'positive', 'sad', 'thanks']
120 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'am', 'anyone', 'are', 'awesome', 'bad', 'be', 'behavior', 'blood', 'bot', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'cool', 'could', 'damn', 'data', 'day', 'detail', 'do', 'doing', 'dont', 'drug', 'entry', 'feeling', 'find', 'fine', 'for', 'funny', 'give', 'good', 'goodbye', 'great', 'happy', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'human', 'humour', 'hurray', 'i', 'id', 'intelligent', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'low', 'management', 'me', 'module', 'name', 'nearby', 'next', 'nice', 'not', 'of', 'offered', 'ok', 'okay', 'open', 'patient', 'pharmacy', 'pressure', 'problem', 'provide', 'reaction', 'relat

In [14]:
pickle.dump(words,open('words1.pkl','wb'))
pickle.dump(classes,open('classes1.pkl','wb'))

In [15]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\SUHAS\AppData\Local\Temp/ipykernel_33304/3528693901.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [16]:
print(train_x)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1

In [17]:
print(train_y)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 

In [18]:
# Create model - 4 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

D:\New folder\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
14/14 [==============================] - 1s 3ms/step - loss: 2.6822 - accuracy: 0.0714 
Epoch 2/200
14/14 [==============================] - 0s 3ms/step - loss: 2.6127 - accuracy: 0.1143
Epoch 3/200
14/14 [==============================] - 0s 3ms/step - loss: 2.5938 - accuracy: 0.1286
Epoch 4/200
14/14 [==============================] - 0s 3ms/step - loss: 2.5144 - accuracy: 0.1571
Epoch 5/200
14/14 [==============================] - 0s 3ms/step - loss: 2.4610 - accuracy: 0.2571
Epoch 6/200
14/14 [==============================] - 0s 3ms/step - loss: 2.4138 - accuracy: 0.2286
Epoch 7/200
14/14 [==============================] - 0s 3ms/step - loss: 2.3979 - accuracy: 0.2286
Epoch 8/200
14/14 [==============================] - 0s 3ms/step - loss: 2.2974 - accuracy: 0.3286
Epoch 9/200
14/14 [==============================] - 0s 3ms/step - loss: 2.2429 - accuracy: 0.3857
Epoch 10/200
14/14 [==============================] - 0s 3ms/step - loss: 2.1333 - accuracy: 0.4571
Epoch 11

In [19]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [31]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))


In [32]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [33]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [34]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [48]:
chatbot_response("you are funny")

1/1 [==============================] - 0s 39ms/step


'Thanks for your compliment'

In [49]:
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
 

base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

1/1 [==============================] - 0s 40ms/step
